In [ ]:
import pandas as pd

df_struct = pd.read_parquet("default_output/structures_metadata.parquet")
df_chain  = pd.read_parquet("default_output/chains_metadata.parquet")
df_role   = pd.read_parquet("default_output/roles_metadata.parquet")
df_iface  = pd.read_parquet("default_output/interfaces_metadata.parquet")
df_allmeta = pd.read_parquet("default_output/all_metadata.parquet")

print(df_struct.columns.tolist())
print(df_chain.columns.tolist())
print(df_role.columns.tolist())
print(df_iface.columns.tolist())


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
from pathlib import Path
import pandas as pd
from atomworks.io import parse

df = pd.read_parquet("out_default/all_metadata.parquet")

top_paths = (
    df.sort_values("iface_antibody__antigen__iface__n_contact_atoms", ascending=False)
      .head(5)["path"]
)

atomarrays = {}
for p in top_paths:
    out = parse(
        filename=p,
        build_assembly=("1",),
        add_missing_atoms=False,
        fix_formal_charges=False,
        add_id_and_entity_annotations=True,
    )
    atomarrays[p] = out["assemblies"]["1"][0]

print("Loaded", len(atomarrays), "structures")


In [ ]:
import biotite.structure as struc

def select_chains(aa, chain_ids):
    mask = False
    for c in chain_ids:
        mask |= (aa.chain_id == c)
    return aa[mask & aa.is_polymer]

aa = atomarrays[top_paths.iloc[0]]
antibody = select_chains(aa, ["H", "L"])
print(len(antibody))


In [ ]:
import shutil

outdir = Path("top_structures")
outdir.mkdir(exist_ok=True)

for p in top_paths:
    shutil.copy(p, outdir / Path(p).name)


In [ ]:
import pandas as pd
from pathlib import Path

df = pd.read_parquet("out_default/all_metadata.parquet")
top = df.sort_values(
    "iface_antibody__antigen__iface__n_contact_atoms",
    ascending=False
).head(10)

with open("load_top10.pml", "w") as f:
    for p in top["path"]:
        f.write(f"load {p}\n")
    f.write("hide everything\nshow cartoon\n")


In [ ]:
row = df.iloc[0]
print(
    row["path"],
    row["iface_antibody__antigen__iface__n_contact_atoms"],
    row["role_antibody__paragraph__paratope_size"],
)


In [ ]:
import pandas as pd

df = pd.read_parquet("out_default/all_metadata.parquet")

df["contact_bin"] = pd.cut(
    df["iface_antibody__antigen__iface__n_contact_atoms"],
    bins=[0, 50, 150, 300, 10000],
    labels=["low", "mid", "high", "very_high"]
)

df["contact_bin"].value_counts()


In [ ]:
# 1. 有没有 path
assert "path" in df.columns

# 2. 有没有你想要的指标
[c for c in df.columns if "iface" in c]

# 3. 有没有 NaN（空 interface / 空 role）
df.isna().mean().sort_values(ascending=False).head(10)

# 4. 有多少 structure 没有 interface
(df["iface_antibody__antigen__iface__n_contact_atoms"].isna()).sum()
